In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!pip install -U sentence-transformers

In [ ]:

from sentence_transformers import SentenceTransformer

In [ ]:
import torch

In [ ]:
from tqdm import tqdm
import nltk
nltk.download('punkt')
from nltk import tokenize

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import transformers

In [ ]:
df = pd.read_csv('../input/dataiitp/DatasetIIT/2018_iclr.csv')
df['confidence'] = df['confidence'].astype(str).str[0]
df['review'] = df['review']
review = df[['confidence','review']]
review = review.dropna(axis=0)

In [ ]:
review['confidence'] = review['confidence'].astype(int) -1 

In [ ]:
reviews = list(review['review'])

In [ ]:
def preprocess_text(sen):
    # Removing html tags
#     sentence = remove_tags(sen)
#     print(type(sen))
#     # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    
    sentence = re.sub(r'\n+', ' ', sentence)
    return sentence

In [ ]:
import regex as re
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [ ]:
sentences = []
for sen in tqdm(reviews):
    sentences.append(preprocess_text(sen.lower()))

In [ ]:
review['review'].str.split().map(lambda x : len(x)).plot.hist(bins=50)

In [ ]:
from transformers import AutoTokenizer, AutoModel

# tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
# model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')



In [ ]:
from sentence_transformers import SentenceTransformer , models



word_embedding_model = models.Transformer( 'bert-large-uncased', max_seq_length = 512) #'allenai/scibert_scivocab_uncased' bert-large-uncased

In [ ]:
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension() , pooling_mode_max_tokens= False , pooling_mode_mean_tokens=False , pooling_mode_mean_sqrt_len_tokens=True)
pooling_model2 = models.Pooling(word_embedding_model.get_word_embedding_dimension() , pooling_mode_max_tokens= False , pooling_mode_mean_tokens=False , pooling_mode_mean_sqrt_len_tokens=True) 

In [ ]:
from torch import nn
dense_models = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=768 , activation_function=nn.Tanh())

In [ ]:
model = SentenceTransformer(modules = [ word_embedding_model , pooling_model])

In [ ]:
embeddings = model.encode(sentences , batch_size=128 , convert_to_numpy = True , show_progress_bar = True)

In [ ]:
X = embeddings
y = np.array(review['confidence'].astype(int))

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_cv,y_train,y_cv = train_test_split(X,y,test_size = 0.2)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense,Conv1D,MaxPooling1D
from keras.layers import LSTM,Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import ModelCheckpoint
# fix random seed for reproducibility
np.random.seed(7)
from prettytable import PrettyTable
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import tensorflow as tf
embedding_vecor_length = 768
model = Sequential()
model.add(Embedding( output_dim = 768, input_dim=768))
model.add(Conv1D(filters=64, kernel_size=8, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32,kernel_size=4,padding="same",activation="relu"))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=16,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=8,kernel_size=1,padding="same",activation="relu"))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(256))
model.add(Dropout(rate=0.4))
model.add(Dense(1, activation='relu'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=[tf.keras.metrics.RootMeanSquaredError() , tf.keras.metrics.MeanAbsoluteError() ])
print(model.summary())
filepath="weights_best_cnn.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val', verbose=1, save_best_only=True, mode='max',save_weights_only=True)
callbacks_list = [checkpoint]
model.fit(X_train,y_train, epochs=20, batch_size=16,verbose = 1,callbacks = callbacks_list,validation_data=(X_cv,y_cv))

In [ ]:
results = model.predict(X_train)
y = y_train




ax1 = sns.distplot(results)
ax2 = sns.distplot(y)
plt.axvline(np.mean(results), color='b', linestyle='dashed', linewidth=5)
plt.axvline(np.mean(y), color='orange', linestyle='dashed', linewidth=5)


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn import model_selection
from matplotlib import pyplot
def metrics(expected , predicted):
  # Mean absolute error
  print("The rmse is " + str(np.sqrt(mean_squared_error(expected, predicted))))
  print("The mae is " + str(mean_absolute_error(expected, predicted)))
  print("The r2 is " + str(r2_score(expected, predicted)))
  print("Standard Deviation of predicted" + str(np.std(predicted)))

metrics(results , y)

In [ ]:
results = model.predict(X_cv)
y = y_cv




ax1 = sns.distplot(results)
ax2 = sns.distplot(y)
plt.axvline(np.mean(results), color='b', linestyle='dashed', linewidth=5)
plt.axvline(np.mean(y), color='orange', linestyle='dashed', linewidth=5)


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn import model_selection
from matplotlib import pyplot
def metrics(expected , predicted):
  # Mean absolute error
  print("The rmse is " + str(np.sqrt(mean_squared_error(expected, predicted))))
  print("The mae is " + str(mean_absolute_error(expected, predicted)))
  print("The r2 is " + str(r2_score(expected, predicted)))
  print("Standard Deviation of predicted" + str(np.std(predicted)))

metrics(results , y)

In [ ]:
y_train

In [ ]:
results